Channels
--------

Part of interaction between codes in OMUSE is based on exchanging data between the *community* codes or exchanging data between these codes and OMUSE. As you might have noticed in the previous tutorial topic, every code provides access to particle collections or grids. The data of these collections or grids *live* inside the code, while the data of collections created in the script *live* inside the python process.


<p style="background-color: lightyellow">
<em>Background:</em> All data storage of particle collections (or grids) is implemented by different storage classes. AMUSE supports storage classes that simply store the data in python lists and numpy arrays. AMUSE also supports storage classes that send messages to the codes to perform the actual storage and retrieval. At the script level the interface to these classes is all the same, so in normal use they behave the same. The performance of the different storage classes will vary, for code storage the data may be sent over an internet connection, causing slower reaction times. Smart usage of channels and caching data in memory sets will increase performance.
</p>

In [ ]:
%matplotlib inline
import numpy
from matplotlib import pyplot
from omuse.units import units, constants
from amuse.datamodel import Particles
from amuse.datamodel import new_regular_grid

It is easy to make two collections with the same properties, we only have to copy the collection

In [ ]:
grid1=new_regular_grid( (3,4), [1.,2.] | units.m)
grid2=grid1.copy()
print grid1
print grid2

Setting the mass of the grid cell in one collection will not influence the cells in the second collection.

In [ ]:
grid1.mass = 1. | units.kg
grid1.area = 1. | units.m**2
print grid1
print grid2

You could however easily copy the data over with an attribute assignment

In [ ]:
grid2.mass = grid1.mass
print grid2

AMUSE provides channels to optimize the transport of attribute values between collections. Channels are also save to use when adding or removing particles in particle sets. Channels are uni-directional, you'll need two to be able to do bi-directional information exchange.

In [ ]:
grid1.mass*=2
channel_from_1_to_2 = grid1.new_channel_to(grid2)
channel_from_1_to_2.copy_attributes(["mass"])
print grid1.mass
print grid2.mass

Channels can be used to copy multiple attributes in one go, this can optimize data transport between codes.

In [ ]:
channel_from_1_to_2.copy_attributes(["mass", "area"])
print grid2

Transforms
----------

In the same way we copy data between grids in the memory of the script we can copy data between grids in memory and in a running simulation code and between simulation codes. Let's illustrate that by using the QGmodel.

In [ ]:
from omuse.community.qgmodel.interface import QGmodel

In [ ]:
q=QGmodel()
q.parameters.dx=40. | units.km
q.parameters.dy=40. | units.km
q.parameters.interface_wind=True

As we can see QGmodel receives windstress forcings (`tau_x` only):

In [ ]:
print q.forcings

Now, suppose that we have a model that generates a grid with wind velocities (here we just generate an empty copy of the forcings grid, and fill that by hand):

In [ ]:
grid=q.forcings.empty_copy()
grid.vx=10. | units.m/units.s
grid.vy=0. | units.m/units.s

Of course there is a functional relation between wind speed and surface wind stress, which we could use: 

In [ ]:
def wind_stress(vx,vy, rho_air=0.0013 | units.g/units.cm**3, winddrag_coeff=0.001):
    v=(vx**2+vy**2)**0.5
    tau_x=rho_air*winddrag_coeff*vx*v
    tau_y=rho_air*winddrag_coeff*vy*v
    return tau_x,tau_y

grid.tau_x=wind_stress(grid.vx,grid.vy)[0]
print grid.tau_x[10,10].in_(units.Pa)
channel=grid.new_channel_to(q.forcings)
channel.copy_attributes(["tau_x"])
print q.forcings[10,10].tau_x.in_(units.Pa)

This is fine, but adds an attribute to grid (which we may not need for anything else). Additionally, we cannot add attributes in a simple way to a grid of a code (remember that in that case the grid refers to storage in the code). An alternative is to use a functional transform:

In [ ]:
grid=q.forcings.empty_copy()
grid.vx=10. | units.m/units.s
grid.vy=0. | units.m/units.s
channel=grid.new_channel_to(q.forcings)
def wind_stress_tau_x(vx,vy):
    return [wind_stress(vx,vy)[0] ] # note that the function must return a list
channel.transform(["tau_x"], wind_stress_tau_x, ["vx","vy"])
print q.forcings[10,10].tau_x.in_(units.Pa)

The transform is specified by (target, function, source), where target and source are lists with the names of the target and source attributes. The function takes as input the arguments specified by source, and must output a list of the same length as target.

Remapping channel
-----------------

In case the source and target grids are not the same shape, or of different type the above channel will not work. 
In this case it is necessary to do a remapping between the values on the grids. In practice this is a situation that
is often encountered when coupling different simulation codes. OMUSE provides a framework for remapping grids.

Within OMUSE one can define a remapping channel between grids, which takes a remapping method as argument. A number of 
remapping methods are available (and these can be used for different source and target grids, although not all 
                                 combinations are implemented yet). We give a simple example using an interpolating 2D remapper.

In [ ]:
from amuse.ext.grid_remappers import interpolating_2D_remapper
from amuse.datamodel import new_cartesian_grid,UnstructuredGrid

We define a simple 2D grid and a 2D grid sampling the domain of the 2D grid:

In [ ]:
grid=new_cartesian_grid((100,100), 1. | units.km)
xc=50. | units.km
yc=50. | units.km
grid.distance=((grid.x-xc)**2+(grid.y-yc)**2)**0.5
smallgrid=new_cartesian_grid((20,20), 0.5 | units.km)
smallgrid.position+=[45.,45.] | units.km

The following defines a channel that can be used to transport values from `grid` to `smallgrid`. The values 
need to be remapped, and for this a simple interpolating remapper is used. 

In [ ]:
channel=grid.new_remapping_channel_to(smallgrid, interpolating_2D_remapper)

Once this channel is defined and initialized, it can be used in much the same way as a normal channel:

In [ ]:
channel.copy_attributes(["distance"])
print smallgrid

In [ ]:
pyplot.subplot(121)
pyplot.imshow(numpy.transpose(grid.distance.value_in(units.km)), origin='lower',interpolation="nearest")
pyplot.subplot(122)
pyplot.imshow(numpy.transpose(smallgrid.distance.value_in(units.km)), origin='lower',interpolation="nearest")

The interpolating remapper can be used for any 2D structured source grid to any target grid with 2D positions. More 
complicated (conservative) remappers are available through the CDO package.  